In [1]:
import tensorflow as tf
import numpy as np
import math

SEPCTURAL_SAMPLES = 10
FEATURE_DIM = SEPCTURAL_SAMPLES*6*2
CONV_LEN = 3
CONV_LEN_INTE = 3#4
CONV_LEN_LAST = 3#5
CONV_NUM = 64
CONV_MERGE_LEN = 8
CONV_MERGE_LEN2 = 6
CONV_MERGE_LEN3 = 4
CONV_NUM2 = 64
INTER_DIM = 128
OUT_DIM = 6#len(idDict)
WIDE = 20
CONV_KEEP_PROB = 0.2

BATCH_SIZE = 64
TOTAL_ITER_NUM = 1000

select = 'a'

metaDict = {'a':[119080, 1193], 'b':[116870, 1413], 'c':[116020, 1477]}
TRAIN_SIZE = metaDict[select][0]
EVAL_DATA_SIZE = metaDict[select][1]
EVAL_ITER_NUM = int(math.ceil(EVAL_DATA_SIZE / BATCH_SIZE))

In [2]:
# ###### Import training data
# train_x = np.load('DeepSense_data/train_x.npy')
# train_y = np.load('DeepSense_data/train_y.npy')
# eval_x = np.load('DeepSense_data/eval_x.npy')
# eval_y = np.load('DeepSense_data/eval_y.npy')

###### Import training data
train_x = np.load('fre_data.npy')
print(train_x.shape)
train_y = np.load('fre_data_label.npy')
# eval_x = np.load('DeepSense_data/eval_x.npy')
# eval_y = np.load('DeepSense_data/eval_y.npy')
train_x = train_x.reshape((-1,20,120))

# train_x = np.array(train_x)
print(train_x.shape,train_y.shape)
# print(eval_x.shape,eval_y.shape)

acc,gyo = np.split(train_x,2)
acc = acc.reshape((-1,20,60,1))
gyo = gyo.reshape((-1,20,60,1))
print(acc.shape,gyo.shape)
print(train_y[0])

(254000, 120)
(12700, 20, 120) (12700, 6)
(12700, 20, 60, 1) (12700, 20, 60, 1)
[0 0 0 0 1 0]


In [3]:
###### Import training data
train_x = np.load('DeepSense_data/train_x.npy')
train_y = np.load('DeepSense_data/train_y.npy')
eval_x = np.load('DeepSense_data/eval_x.npy')
eval_y = np.load('DeepSense_data/eval_y.npy')

# train_x = np.array(train_x)
print(train_x.shape,train_y.shape)
print(eval_x.shape,eval_y.shape)
# train_x = np.expand_dims(train_x, axis=3)
# acc,gyo = np.split(train_x,2,axis=2)
# acc = acc.reshape((-1,20,60,1))
# gyo = gyo.reshape((-1,20,60,1))
# print(acc.shape,gyo.shape)
print(train_y[0])

(119080, 20, 120) (119080, 6)
(1193, 20, 120) (1193, 6)
[0. 0. 0. 0. 1. 0.]


In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [4]:
# Input Data 

training_data_count = len(x_train)  # 7352 training series (with 50% overlap between each serie)
#test_data_count = len(eval_x)  # 2947 testing series
n_steps = len(x_train[0])  # 128 timesteps per series
n_input = len(x_train[0][0])  # 9 input parameters per timestep


# train_x = (train_x - np.min(train_x))/np.mean(train_x)
print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(train_x.shape, train_y.shape, np.mean(x_train), np.std(x_train))

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(12700, 20, 120) (12700, 6) 4.992907516420385e-19 5.345805606954511


In [4]:
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.core import Activation
from keras.layers import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv3D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras import optimizers
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Reshape
import keras.backend as K
from keras.layers.recurrent import GRU,LSTM
from keras.layers import GRUCell,Lambda
from keras.layers import RNN
from keras import regularizers

Using TensorFlow backend.


In [5]:
n_hidden = 64 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0005
lambda_loss_amount = 0.0005
#training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 128
display_iter = 30000  # To show test set accuracy during training


sensor_input = Input(shape=(20,120), name='input')
outputs = LSTM(n_hidden,unit_forget_bias=True,return_sequences=True)(sensor_input)
outputs = LSTM(n_hidden,unit_forget_bias=True,return_sequences=True)(outputs)

def reduce_dim(x):
    return x[:,-1:,:]

outputs = Lambda(reduce_dim)(outputs)
outputs = Activation('linear')(outputs)
outputs = Flatten()(outputs)

outputs = Dense(6, activation='softmax',kernel_regularizer=regularizers.l2(lambda_loss_amount))(outputs)
# keras.regularizers.l2(lambda_loss_amount)
adam = keras.optimizers.Adam(lr=learning_rate,epsilon=None, decay=0.0, amsgrad=False)

model = Model(inputs=sensor_input, outputs=outputs)
model.compile(loss='categorical_crossentropy',
      optimizer=adam,
      metrics=['accuracy'])

#model.fit([acc_inputs,gyro_inputs], train_y, batch_size=64, epochs=10)
model.summary()


W0917 16:59:49.346907  9748 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0917 16:59:49.366908  9748 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0917 16:59:49.368908  9748 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0917 16:59:49.918940  9748 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimize

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 20, 120)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 64)            47360     
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 64)            33024     
_________________________________________________________________
lambda_1 (Lambda)            (None, 1, 64)             0         
_________________________________________________________________
activation_1 (Activation)    (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total para

In [ ]:
model.fit(train_x, train_y, batch_size=64,validation_split=0.2, epochs=20)

W0917 16:59:54.422197  9748 deprecation.py:323] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0917 16:59:56.043290  9748 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 95264 samples, validate on 23816 samples
Epoch 1/20
95264/95264 [==============================] - 241s 3ms/step - loss: 0.1629 - acc: 0.9456 - val_loss: 0.1406 - val_acc: 0.9694
Epoch 2/20
95264/95264 [==============================] - 234s 2ms/step - loss: 0.0285 - acc: 0.9943 - val_loss: 0.1304 - val_acc: 0.9650
Epoch 3/20
12800/95264 [===>..........................] - ETA: 3:04 - loss: 0.0199 - acc: 0.9973

In [9]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn import metrics
test_x = x_test
test_y = y_test

# print ( "Loss = " + str( preds[0] ) )
# print ( "Test Accuracy = " + str( preds[1] ) )
print(model.metrics_names)
loss, acc = model.evaluate(test_x, test_y, batch_size=128)
print('loss, acc ',loss,acc)


y_pred = model.predict(test_x, batch_size=128, verbose=1)
# y_pred_bool = np.argmax(preds, axis=1)

for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0

print(classification_report(test_y, y_pred))
print(metrics.f1_score(test_y, y_pred, average="weighted"))
# print(confusion_matrix(test_y.argmax(axis=1), y_pred))

['loss', 'acc']
2540/2540 [==============================] - 1s 368us/step
loss, acc  0.10084378505785634 0.9724409443186963
2540/2540 [==============================] - 1s 356us/step
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       386
           1       1.00      1.00      1.00       500
           2       0.89      0.99      0.94       358
           3       0.97      0.95      0.96       399
           4       0.99      1.00      0.99       417
           5       1.00      0.93      0.96       480

   micro avg       0.97      0.97      0.97      2540
   macro avg       0.97      0.97      0.97      2540
weighted avg       0.97      0.97      0.97      2540
 samples avg       0.97      0.97      0.97      2540

0.9726304060305165


In [12]:
print(metrics.f1_score(test_y, y_pred, average="micro"))

0.9724409448818898
